<a href="https://colab.research.google.com/github/louiezzang/next-gpt/blob/main/examples/huggingface_sft_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Supervised Fine-tuning with huggingface


Build a Supervised Fine-tuning model to answer well to the question.

- SFT(Supervised Fine Tuning)
- Fine-tune a pretrained LLM on a specific domain or corpus of instructions and human demonstrations

- Dataset example
```json
[
    {
        "prompt": "",
        "completion": ""        
    }, ...
]
```

# Environment setup

#### Installation (python>=3.8)

In [1]:
# Install next-gpt lib.
!rm -rf ./next-gpt/
!git clone https://github.com/louiezzang/next-gpt.git
%cd next-gpt/
!pip install .
%cd ../

Cloning into 'next-gpt'...
remote: Enumerating objects: 801, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 801 (delta 20), reused 26 (delta 10), pack-reused 745
Receiving objects: 100% (801/801), 280.41 KiB | 18.69 MiB/s, done.
Resolving deltas: 100% (461/461), done.
/content/next-gpt
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/next-gpt
  Preparing metadata (setup.py) ... done
  Created wheel for nextgpt: filename=nextgpt-0.1.0-py3-none-any.whl size=51959 sha256=29cc75f06d64e867ffc2403093a3fe997e548f398f3b06ce2dfde4555d9f9095
  Stored in directory: /root/.cache/pip/wheels/f4/0d/a2/307ec9a6214260bfd63facee827d7bbef5c1ba9618277ea1b5
Successfully built nextgpt
  Attempting uninstall: nextgpt
    Found existing installation: nextgpt 0.1.0
    Uninstalling nextgpt-0.1.0:
      Successfully uninstalled nextgpt-0.1.0
/content


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import json
import yaml
import argparse

import numpy as np
import pandas as pd

import torch
from datasets import load_dataset
import transformers

from transformers import (
    AutoTokenizer, AutoConfig, AutoModelForCausalLM, pipeline, 
    TrainingArguments, AutoModelWithLMHead,
    ProgressCallback
)
from nextgpt.dataset import (
    SupervisedDataset, DataCollatorForSupervisedDataset
)
from nextgpt.finetuning import (
    SupervisedTrainer, LoggingCallback
)

In [4]:
# Define arguments.
parser = argparse.ArgumentParser()
parser.add_argument("--model", type=str, default="gpt2", choices=["gpt2", "bloom", "opt"])
parser.add_argument("--max_epochs", type=int, default=1)
parser.add_argument("--train_batch_size", type=int, default=4)
parser.add_argument("--output_dir", type=str, default="./output_1_sft")

args = parser.parse_args(args=[])
print(args)

Namespace(model='gpt2', max_epochs=1, train_batch_size=4, output_dir='./output_1_sft')


In [5]:
# Get the tokenizer.
tokenizer = transformers.AutoTokenizer.from_pretrained(args.model, 
                                        #   bos_token="<|startoftext|>",
                                        #   eos_token="<|endoftext|>", 
                                        #   pad_token="<|pad|>"
                                          )
tokenizer.pad_token = tokenizer.eos_token
print(tokenizer)

GPT2TokenizerFast(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'})


In [6]:
dataset_webgpt_comp = load_dataset("openai/webgpt_comparisons", split="train[:20%]")

Generating train split: 0 examples [00:00, ? examples/s]

Dataset webgpt_comparisons downloaded and prepared to /root/.cache/huggingface/datasets/openai___webgpt_comparisons/default/0.0.0/8b5d5879cdc98c4c0099af6053dffe8d504588d43d3b11f1b1ec223ab1e8db0a. Subsequent calls will reuse this data.


In [7]:
data_list = []
for row in dataset_webgpt_comp:
  question = row["question"]["full_text"]
  answer_0 = row["answer_0"]
  data_list.append({
      "instruction": question,
      "completion": answer_0
  })

In [8]:
PROMPT_TEMPLATE = (
  "Below is an instruction that describes a task, paired with an input that provides further context. "
  "Write a response that appropriately completes the request.\n\n"
  "### Instruction:\n{instruction}\n\n### Response:"
)

In [10]:
dataset = SupervisedDataset(
    dataset=data_list,
    tokenizer=tokenizer, 
    prompt_template=PROMPT_TEMPLATE,
    completion_field="completion",
    verbose=True)

# Split train and val dataset.
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, eval_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Data collator.
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

Loading data...
Formatting inputs...
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Voiced by Harry Shearer, what Simpsons character was modeled after Ted Koppel?

### Response:
The Simpsons character that was possibly based on Ted Koppel is Kent Brockman.  He is a local news anchor in Springfield and is modeled after Ted Koppel. [1]<|endoftext|>
Tokenizing inputs... This may take some time...


In [11]:
# Load the pretrained model.
model = AutoModelForCausalLM.from_pretrained(args.model)
model.resize_token_embeddings(len(tokenizer))

Embedding(50257, 768)

In [12]:
# Train arguments.
training_args = TrainingArguments(
    output_dir="./train_1_sft", # the output directory
    overwrite_output_dir=True, # overwrite the content of the output directory
    num_train_epochs=args.max_epochs, # number of training epochs
    per_device_train_batch_size=args.train_batch_size, # batch size for training
    per_device_eval_batch_size=4, # batch size for evaluation
    eval_steps=3, # number of update steps between two evaluations.
    save_steps=100, # after # steps model is saved 
    warmup_steps=5, # number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
)

# Train the model.
trainer = SupervisedTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=None,
    # callbacks=[ProgressCallback, LoggingCallback(logger=None)],
)

trainer.train()
trainer.save_state()
trainer.safe_save_model(output_dir=args.output_dir)

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,2.782900


In [13]:
# Inference test.
generator = pipeline("text-generation", model=args.output_dir, tokenizer=tokenizer)

generation_args = dict(
    num_beams=4,
    repetition_penalty=2.0,
    no_repeat_ngram_size=4,
    # bos_token="<|startoftext|>",
    # eos_token="<|endoftext|>", 
    # pad_token="<|pad|>",
    max_new_tokens=64,
    do_sample=True,
    top_k=30,
    top_p=0.95,
    temperature=1.9, 
    #max_length=300, 
    #num_return_sequences=20
    early_stopping=True,
)

In [14]:
test_list = data_list[-5:]

test_prompt_list = []
actual_completion_list = []
for row in test_list:
    text_input = row
    prompt = PROMPT_TEMPLATE.format_map(text_input)
    test_prompt_list.append(prompt)
    actual_completion_list.append(text_input["completion"])

result_list = generator(test_prompt_list, **generation_args)
for prompt, result, actual_response in zip(test_prompt_list, result_list, actual_completion_list):
    print("")
    print("-" * 70)
    print(("completion: %s" % (result[0]["generated_text"])))
    print(f"\n### Actual answer:\n{actual_response}")

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



----------------------------------------------------------------------
completion: Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
I've noticed when scanning my files for malware/viruses, the "number of files scanned" that pops up is almost always greater than the number of files I selected to scan. What is actually being scanned and why is it considered different files?

### Response:The average number of files scanned depends on the size of your computer's hard drives. If you have 4 GB or smaller hard drive, there are approximately 2 million file scans each month [1]. Additionally, some computers will be able to scan multiple files at once [1]. In this scenario, if more than one file

### Actual answer:
Microsoft Defender Antivirus has multiple layers of protection to catch malware and viruses. These include quick scans, full scans, and on-access protection